# Recurrent Neural Networks
* In this demo, we will apply the recurrent neural network on the Sunspot dataset (https://www.kaggle.com/robervalt/sunspots)
* The code in this demo is adapted from: https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/

## 1. Read dataset into a Pandas dataframe and inspect it
* We use Pandas to read the csv data.
* The data contain the month and average sunspots in that month.

In [ ]:
from pandas import read_csv

sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
df = read_csv(sunspots_url)
print(df)

## 2. Scale the feature values to (0, 1)
* We use MinMaxScaler to scale the feature values to (0, 1). 
* The API for MinMaxScaler is at: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
data = np.array(df['Sunspots'], dtype='float32').reshape((-1, 1))
data = scaler.fit_transform(data).flatten()

print(data.shape)
print(data)

## 3. Split the time series data into train/test sets
* We use the first part of the time series for training, and the remaining part for testing.
* The variable **split_ratio** specifies the ratio of the train split. 

In [ ]:
split_ratio = 0.8 # Percentage to use for training

split_index = int(len(data)*split_ratio) # Index to split train/test sets
train_data = data[:split_index]
test_data = data[split_index:]

print(train_data.shape, test_data.shape)

## 4. Visualize the train/test splits
* We now plot the data, which include both the train and test splits.
* The train and test splits are separated by the red line.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6), dpi=80)
plt.plot(data)
plt.axvline(x=split_index, color='r')
plt.xlabel('Number of months from 1749/01')
plt.ylabel('Number of sunspots (scaled)')
plt.title('Train and test split separated by the red line.')
plt.show()

## 5. Prepare inputs and targets for each split
* Now we need to prepare the inputs and target for trainin and testing.
* We will apply a sliding window with length **time_steps** to the time series. When applying the sliding window at a position, we will use *time_steps* values in the sliding window to predict the next value in the series.
* Then we move the sliding window to the right by **sliding_steps**.
* The function **get_XY** below performs these operations.

In [ ]:
# Function to prepare the input X and target Y
def get_XY(dat, time_steps, sliding_steps):
    # Prepare the targets Y
    Y_ind = np.arange(time_steps, len(dat), sliding_steps)
    Y = dat[Y_ind]
    
    # Prepare the inputs X
    X = []
    for i in Y_ind:
        X.append(dat[i-time_steps:i])
    X = np.array(X).reshape((len(Y), time_steps, 1))
    
    return X, Y

time_steps = 12
sliding_steps = 12
trainX, trainY = get_XY(train_data, time_steps, sliding_steps)
testX, testY = get_XY(test_data, time_steps, sliding_steps)

print(trainX.shape, trainY.shape, testX.shape, testY.shape)

## 6. Define and train RNN model
* We define an RNN model with a SimpleRNN layer with 3 hidden nodes, then a Dense layer with 1 hidden node (for regression).
* We train the model using MSE loss.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

model = Sequential()
model.add(SimpleRNN(units=3, input_shape=(time_steps,1), activation='tanh'))
model.add(Dense(units=1, activation='tanh'))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=20, batch_size=5)

## 7. Evaluate the trained model
* We make predictions and evaluate our model using MSE.

In [ ]:
from sklearn.metrics import mean_squared_error

train_predict = model.predict(trainX)
test_predict = model.predict(testX)

train_mse = mean_squared_error(trainY, train_predict)
test_mse = mean_squared_error(testY, test_predict)

print('Train MSE: %.3f' % (train_mse))
print('Test MSE: %.3f' % (test_mse))

## 8. Plot the true targets and predictions
* We plot the true and predicted target values.

In [ ]:
true_targets = np.append(trainY, testY)
predicted_targets = np.append(train_predict, test_predict)

plt.figure(figsize=(15, 6), dpi=80)
plt.plot(true_targets)
plt.plot(predicted_targets)
plt.axvline(x=len(trainY), color='r')
plt.legend(['True target', 'Predicted target'])
plt.xlabel('Target index')
plt.ylabel('Sunspots scaled')
plt.title('Actual and predicted target values. The red line separates the training and test examples.')
plt.show()